Use [RISE](https://github.com/damianavila/RISE) to display the slides.

```bash
pip install RISE
```

## Dokumentide vektoresitus

#### Notebook colabis: https://tinyurl.com/info-eraldus-4

In [ ]:
import csv

# Loeme sisse näidisandmestiku
texts = []
with open("data/text_paragraphs.csv", "r") as fin:
    reader = csv.reader(fin)
    for row in reader:
        texts.append(row[0])
texts[0]        

In [ ]:
# Kasutame vektoresituse leidmiseks teeki sklearn.
# Vajadusel installige oma kasutatavasse conda keskkonda käsuga 
# conda install scikit-learn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = CountVectorizer()

In [ ]:
t = ['Kured söövad konni.', 'Konni söövad ka luiged ja söövad ka pardid.']
matrix = vectorizer.fit_transform(t)

In [ ]:
matrix.shape

In [ ]:
matrix.todense()

In [ ]:
vocab = vectorizer.get_feature_names()
vocab

In [ ]:
# Meetod teksti sõnestamiseks ja lemmatiseerimiseks estnltk abil
from estnltk import Text
def lemmatize_with_estnltk(input_string):
    lemmas = []
    text = Text(input_string).tag_layer()
    for word in text.morph_analysis:
        lemmas.append(word.lemma[0])
    return lemmas

vectorizer = TfidfVectorizer(tokenizer = lemmatize_with_estnltk)

In [ ]:
# Andmete viimine vektorkujule
matrix = vectorizer.fit_transform(texts)
matrix.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

# Leiame kõigi tekstide kaugused üksteisest
cos_dist = cosine_distances(matrix)
cos_dist

In [ ]:
# Uurime tekste, mis on omavahel mõnevõrra sarnased
for idx, k in enumerate(cos_dist):
    for idx2, j in enumerate(k):
        if idx2 > idx:
            if j > 0.4 and j < 0.6:
                print(j, '\n\n', texts[idx], '\n\n', texts[idx2], '\n', '-'*40, '\n')

## Tekstide klassifitseerimine

In [ ]:
# Loome oma tekstidest "märgendatud" andmestiku
labels = []
for text in texts:
    if 'koroona' in text.lower() or 'corona' in text.lower() \
    or 'covid' in text.lower() or 'haig' in text.lower():
        labels.append(1)
    else:
        labels.append(0)

In [ ]:
# Loome uue vektoriseerija, mis vaataks ka 2-3-sõnalisi järjendeid
vectorizer = TfidfVectorizer(ngram_range = (1,3), tokenizer = lemmatize_with_estnltk)
corpus = vectorizer.fit_transform(texts)
vocab = vectorizer.get_feature_names()

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split

# Jagame loodud andmestiku treening- ja testandmeteks
x_train, x_test, y_train, y_test, desc_train, desc_test = train_test_split(corpus, labels, texts, random_state = 1)
lin_clf = svm.LinearSVC()
# Treenime mudeli
lin_clf.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import f1_score
# Laseme mudelil ennustada testandmete klassid
pred = lin_clf.predict(x_test)
# Võrdleme klasse tegelikkusega - kui palju läks täppi?
f1 = f1_score(y_test, pred)
f1

In [ ]:
# Uurime vigu - millised tekstid läksid valesse klassi?
idx = 0
for real, predicted in zip(y_test, pred):
    if real != predicted:
        if real == 1:
            print(desc_test[idx])
            print('--------------------')
    idx += 1    

In [ ]:
# Mille põhjal mudel otsustas?
import numpy as np
top20 = np.argsort(lin_clf.coef_[0])[-20:]
print("%s" % (" ; ".join(vocab[j] for j in top20)))

## Teemade modelleerimine

In [ ]:
# Loome uue vektoriseerija - LDA toimib sõnasageduste pealt
vectorizer = CountVectorizer(tokenizer = lemmatize_with_estnltk)
matrix = vectorizer.fit_transform(texts)

In [ ]:
# Kasutame teeki gensim - vajadusel installige käsuga
# conda install gensim
from gensim.models.ldamulticore import LdaModel
from gensim.matutils import Sparse2Corpus
corpus = Sparse2Corpus(matrix, documents_columns = False)

In [ ]:
vocab = np.array(vectorizer.get_feature_names())

# Loome sobival kujul sõnastiku
id2word = {}
for idx, i in enumerate(vocab):
    id2word[idx] = i

In [ ]:
lda = LdaModel(corpus, num_topics=10, id2word=id2word)

In [ ]:
# Vaatame leitud teemasid
lda.show_topics(10, 20)

In [ ]:
# Leiame, millisesse teemasse mingi dokument suurima tõenäosusega kuulub
assignments = lda.inference(corpus)
# argmax() leiab suurima väärtusega elemendi indeksi 
# ning axis = 1 täpsustab, et vaatame rea kaupa
most_likely_labels = assignments[0].argmax(axis=1)

In [ ]:
most_likely_labels

In [ ]:
for k, l in zip(texts, most_likely_labels):
    if l == 4:
        print(k)
        print('----------------')

<img src="pildid/dist_sem1.png">

<img src="pildid/dist_sem2.png">

### Distributiivne semantika - proovime järgi

In [ ]:
# Impordime vajalikud moodulid
import csv
from gensim.models import Word2Vec, KeyedVectors
from estnltk import Text
from collections import Counter

In [ ]:
# model from here http://193.40.33.66/pretrained/cbow_100_5_10_20.zip
# documentation here https://github.com/eleriaedmaa/embeddings

# mudelifaili tee vaja asendada sellega, kuhu mudeli alla laadisite oma arvutis
model = KeyedVectors.load_word2vec_format("../../../AREA1/cbow_100_5_10_20/ettenten.txt.word_vectors", binary=False)

In [ ]:
most_similar = model.most_similar(positive=['kurk', 'porgand'], topn=100)

In [ ]:
most_similar[:20]

In [ ]:
most_similar = model.most_similar(positive=['müüja', 'dirigent', 'ehitaja'], topn=100)

In [ ]:
most_similar[:20]

<img src="pildid/baas2.png">

### Oma nimeüksuste lisamine

1) Leiame võimalikult suure hulga näiteid vastavast nimeüksusest
 
2) Märgendame ümber olemasoleva korpuse - anname seal esinevatele vastavatele nimeüksustele soovitud märgendid

3) Treenime nimeüksuste tuvastaja ümbermärgendatud korpuse peal

4) Märgendame oma nimeüksusi

In [ ]:
from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.refac.ner import ModelStorageUtil
from estnltk.core import DEFAULT_PY3_NER_MODEL_DIR
from estnltk.taggers import NerTagger

ner_tagger = NerTagger()
model_dir=DEFAULT_PY3_NER_MODEL_DIR
modelUtil = ModelStorageUtil(model_dir)
nersettings = modelUtil.load_settings()
trainer = NerTrainer(nersettings)

In [ ]:
# See on meie treeningkorpus - peab olema morfoloogiliselt märgendatud
text = Text('Eesti president käis Euroopas.').tag_layer()

In [ ]:
text.words

In [ ]:
# Igale sõnale peab vastama märgend; iga lause kohta üks list
labels = [['B-LOC','B-VOC','O','B-LOC', 'O']]

In [ ]:
# Treenime, kasutades treeningkorpust ja märgendeid;
# salvestame tulemuse kausta 'test'
trainer.train(text, labels, 'test')

In [ ]:
# See on meie testkorpus - ei kattu treeningkorpusega
text2 = Text('Läti president elab Riias.').tag_layer()

In [ ]:
# Standardne NER-tagger leiab asukohad
ner_tagger.tag(text2)

In [ ]:
text2.ner

In [ ]:
# Loome uue NER-taggeri oma loodud 'test' mudeli põhjal
# Anname väljundkihile nimeks ner2, 
# sest ner kiht on juba meie tekstil olemas
ner_tagger2 = NerTagger(model_dir = 'test', output_layer = 'ner2')

In [ ]:
ner_tagger2.tag(text2)

In [ ]:
text2.ner

In [ ]:
text2.ner2

In [ ]:
text3 = Text('Läti peaminister elab ka Riias.').tag_layer()
ner_tagger2.tag(text3)
text3.ner2

<img src="pildid/kokkuvõte1.png">

<img src="pildid/kokuvõte2.png">

<img src="pildid/kokkuvõte.png">

<img src="pildid/lõpp.png">